<a href="https://colab.research.google.com/github/gabrielcarcedo/Taller-Pre-Congreso-CNIB-2025-ML-en-Datos-Medicos/blob/main/Notebooks/Taller_Pre_Congreso_NLP_medical_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine Learning para Procesamiento de Datos Médicos**
---
## Taller Pre-Congreso CNIB 2025
---
<table>
  <tr>
      <h4><b>ISC. Gabriel Carcedo Rodríguez</b> (<i>PCIC - IIMAS, UNAM</i>): <a href= "mailto:gabrielcarcedo@comunidad.unam.mx">gabrielcarcedo@comunidad.unam.mx</a>
      <br>
      <br>
      <b>Ing. Ismael Pérez Ruiz</b> (<i>Universidad Modelo</i>): <a href= "mailto:CORREO">CORREO</a>
      <br>
      <br>
      <b>LCC. Victoria May Balam</b> (<i>PCIC - IIMAS, UNAM</i>): <a href= "mailto:victoria.mbalam@comunidad.unam.mx">victoria.mbalam@comunidad.unam.mx</a> </h4>
  </tr>
  <tr>
    <td width="60%" valign="top">
      <!-- Content for Column 1 -->
      <br>
      <br>
      <a href="https://www.pcic.unam.mx/">Posgrado en Ciencia e Ingeniería de la Computación</a>
    </td>
    <td width="40%" valign="top" style="text-align: center;">
      <!-- Content for Column 2 -->
      <a href="https://www.pcic.unam.mx/">
        <img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="PCIC" style="width:100%; height:auto;">
      </a>
    </td>
  </tr>
  <tr>
    <td width="60%" valign="top">
      <!-- Content for Column 1 -->
      <br>
      <br>
      <a href="https://www.unimodelo.edu.mx/merida">Universidad Modelo</a>
    </td>
    <td width="40%" valign="top" style="text-align: center;">
      <!-- Content for Column 2 -->
      <a href="https://www.unimodelo.edu.mx/merida">
        <img src="https://servicios.unimodelo.edu.mx/merida/ing/practicas/resources/imgs/logo-modelo.png" alt="Universidad Modelo" style="width:100%; height:auto;">
      </a>
    </td>
  </tr>

</table>


Nota: Libreta inspirada en la clase de NLP en medical reports de la Dra. Helena Gómez Adorno y Diego Hernández Bustamante durante la Escuela Híbrida de Verano "Medical Informatics with Artificial Intelligence" *UNAM-TUBS*

In [ ]:
#@markdown # Install spaCy and Download Spanish Language Model
!pip -q install spacy
!python -m spacy download es_core_news_md

In [ ]:
# Using short Electronic Medical Report (EMR)
emr = [
"""Paciente mujer de 45 años. Niega fiebre. Refiere dolor torácico desde hace 2 días y disnea leve.
Se indica paracetamol 500 mg cada 8 h. Examen: crepitantes en base pulmonar derecha.""",
"""Varón de 62 años con DM2 en control. Sin tos ni cefalea. Se prescribe metformina 850 mg 2 veces al día.
Dolor abdominal intermitente; descarta vómito. Amoxicilina 500 mg c/12h por 7 días.""",
"""Paciente masculino 30 años. Faringitis; ibuprofeno 400mg cada 8h. Niega disnea. Ausencia de fiebre.
Dolor en garganta y adenopatías cervicales."""
]

# Building a simple **Named Entity Recognition (NER)** system without using machine learning models

In [ ]:
# Import regex for pattern matching, json for data handling
import re, json

# Import spaCy for natural language processing
import spacy

# Import EntityRuler for custom entity recognition patterns
from spacy.pipeline import EntityRuler

# Load Spanish language model (medium size, excluding named entity recognition)
nlp = spacy.load("es_core_news_md",  exclude=["ner"])

In [ ]:
# Define medical terminology lists

MEDICAMENTOS = ["paracetamol","metformina","amoxicilina","ibuprofeno","naproxeno","omeprazol"]  # Medications

SINTOMAS = ["fiebre","tos","cefalea","disnea","vómito","dolor","faringitis","adenopatías","náusea"]  # Symptoms

ANATOMIA = ["pulmonar","pulmón","torácico","garganta","abdomen","cervicales","corazón"]  # Anatomy terms

In [ ]:
# Helper function to create entity patterns from word lists

def dict_patterns(terms, label):
    return [{"label": label, "pattern": [{"LOWER": t.lower()}]} for t in terms]

## Patterns List

In [ ]:
# Initialize patterns list
patterns = []

In [ ]:
# Add medication patterns
patterns += dict_patterns(MEDICAMENTOS, "MEDICAMENTO")
patterns

[{'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'paracetamol'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'metformina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'amoxicilina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'ibuprofeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'naproxeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'omeprazol'}]}]

In [ ]:
# Add dosage patterns (e.g., "500 mg")
patterns += [{"label":"DOSIS", "pattern": [{"TEXT":{"REGEX": r"^\d+(\.\d+)?$"}},{"LOWER":{"IN":["mg","ml","mcg"]}}]}]
patterns

[{'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'paracetamol'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'metformina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'amoxicilina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'ibuprofeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'naproxeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'omeprazol'}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?$'}},
   {'LOWER': {'IN': ['mg', 'ml', 'mcg']}}]}]

In [ ]:
# Add dosage patterns without space (e.g., "400mg")
patterns += [{"label":"DOSIS", "pattern": [{"TEXT":{"REGEX": r"^\d+(\.\d+)?(mg|ml|mcg)$"}}]}]
patterns

[{'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'paracetamol'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'metformina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'amoxicilina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'ibuprofeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'naproxeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'omeprazol'}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?$'}},
   {'LOWER': {'IN': ['mg', 'ml', 'mcg']}}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?(mg|ml|mcg)$'}}]}]

In [ ]:
# Add frequency patterns (e.g., "cada 8 horas")
patterns += [{"label":"FRECUENCIA", "pattern": [{"LOWER":{"IN":["cada","c/","veces"]}}, {"IS_ALPHA": True, "OP":"*"}]}]
patterns

[{'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'paracetamol'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'metformina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'amoxicilina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'ibuprofeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'naproxeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'omeprazol'}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?$'}},
   {'LOWER': {'IN': ['mg', 'ml', 'mcg']}}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?(mg|ml|mcg)$'}}]},
 {'label': 'FRECUENCIA',
  'pattern': [{'LOWER': {'IN': ['cada', 'c/', 'veces']}},
   {'IS_ALPHA': True, 'OP': '*'}]}]

In [ ]:
# Add symptom patterns
patterns += dict_patterns(SINTOMAS, "SINTOMA")
patterns

[{'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'paracetamol'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'metformina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'amoxicilina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'ibuprofeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'naproxeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'omeprazol'}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?$'}},
   {'LOWER': {'IN': ['mg', 'ml', 'mcg']}}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?(mg|ml|mcg)$'}}]},
 {'label': 'FRECUENCIA',
  'pattern': [{'LOWER': {'IN': ['cada', 'c/', 'veces']}},
   {'IS_ALPHA': True, 'OP': '*'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'fiebre'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'tos'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'cefalea'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'disnea'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'vómito'}]},


In [ ]:
# Add anatomy patterns
patterns += dict_patterns(ANATOMIA, "ANATOMIA")
patterns

[{'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'paracetamol'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'metformina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'amoxicilina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'ibuprofeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'naproxeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'omeprazol'}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?$'}},
   {'LOWER': {'IN': ['mg', 'ml', 'mcg']}}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?(mg|ml|mcg)$'}}]},
 {'label': 'FRECUENCIA',
  'pattern': [{'LOWER': {'IN': ['cada', 'c/', 'veces']}},
   {'IS_ALPHA': True, 'OP': '*'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'fiebre'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'tos'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'cefalea'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'disnea'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'vómito'}]},


In [ ]:
# Add specific finding pattern
patterns += [{"label":"HALLAZGO","pattern":"crepitantes"}]  # Medical finding: "crepitantes" (crackles)
patterns

[{'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'paracetamol'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'metformina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'amoxicilina'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'ibuprofeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'naproxeno'}]},
 {'label': 'MEDICAMENTO', 'pattern': [{'LOWER': 'omeprazol'}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?$'}},
   {'LOWER': {'IN': ['mg', 'ml', 'mcg']}}]},
 {'label': 'DOSIS',
  'pattern': [{'TEXT': {'REGEX': '^\\d+(\\.\\d+)?(mg|ml|mcg)$'}}]},
 {'label': 'FRECUENCIA',
  'pattern': [{'LOWER': {'IN': ['cada', 'c/', 'veces']}},
   {'IS_ALPHA': True, 'OP': '*'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'fiebre'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'tos'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'cefalea'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'disnea'}]},
 {'label': 'SINTOMA', 'pattern': [{'LOWER': 'vómito'}]},


## Entity Ruler to spaCy pipeline

In [ ]:
# Add custom entity ruler to spaCy pipeline, overwriting existing entities
ruler = nlp.add_pipe("entity_ruler", config={"overwrite_ents": True})

# Register all defined patterns with the entity ruler
ruler.add_patterns(patterns)

print(ruler)

# Process each document in the **'emr'** collection

In [ ]:
for i, txt in enumerate(emr, 1):  # Start enumeration at 1
    # Apply NLP pipeline to text
    doc = nlp(txt)

    # Print document header
    print(f"\n--- DOC {i} ---\n{txt}\n")

    # Print all identified entities with their positions
    for ent in doc.ents:
      print(f"{ent.text} ({ent.label_}) [{ent.start_char}, {ent.end_char}]")


--- DOC 1 ---
Paciente mujer de 45 años. Niega fiebre. Refiere dolor torácico desde hace 2 días y disnea leve.
Se indica paracetamol 500 mg cada 8 h. Examen: crepitantes en base pulmonar derecha.

fiebre (SINTOMA) [33, 39]
dolor (SINTOMA) [49, 54]
torácico (ANATOMIA) [55, 63]
disnea (SINTOMA) [84, 90]
paracetamol (MEDICAMENTO) [107, 118]
500 mg (DOSIS) [119, 125]
cada (FRECUENCIA) [126, 130]
crepitantes (HALLAZGO) [144, 155]
pulmonar (ANATOMIA) [164, 172]

--- DOC 2 ---
Varón de 62 años con DM2 en control. Sin tos ni cefalea. Se prescribe metformina 850 mg 2 veces al día.
Dolor abdominal intermitente; descarta vómito. Amoxicilina 500 mg c/12h por 7 días.

tos (SINTOMA) [41, 44]
cefalea (SINTOMA) [48, 55]
metformina (MEDICAMENTO) [70, 80]
850 mg (DOSIS) [81, 87]
veces al día (FRECUENCIA) [90, 102]
Dolor (SINTOMA) [104, 109]
vómito (SINTOMA) [143, 149]
Amoxicilina (MEDICAMENTO) [151, 162]
500 mg (DOSIS) [163, 169]

--- DOC 3 ---
Paciente masculino 30 años. Faringitis; ibuprofeno 400mg c

# Enhancement the extracted information with context and relations

In [ ]:
# Negation detection: Identify if a symptom or finding is explicitly negated (e.g., “niega fiebre” → Fever = NEGATED).

NEG_CUES = ["no", "sin", "niega", "descarta", "ausencia de", "niega tener", "no refiere"]

In [ ]:
def detect_negation(doc, NEG_CUES, target_labels=("SINTOMA","HALLAZGO","PROBLEMA")):
    results = []
    for ent in doc.ents:
        if ent.label_ in target_labels:
            # Look back up to 5 tokens in the same sentence for a negation cue
            sent = ent.sent
            window = doc[ max(sent.start, ent.start-5) : ent.start ].text.lower()
            negated = any(cue in window for cue in NEG_CUES)
            results.append({
                "text": ent.text,
                "label": ent.label_,
                "start": ent.start_char,
                "end": ent.end_char,
                "negated": negated
            })
    return results

In [ ]:
def link_med_dose(doc, max_window=8):
    pairs = []
    meds = [e for e in doc.ents if e.label_=="MEDICAMENTO"]
    doses = [e for e in doc.ents if e.label_=="DOSIS"]
    freqs = [e for e in doc.ents if e.label_=="FRECUENCIA"]
    for med in meds:
        # choose nearest dose in same sentence within window
        cand = [(dose, abs(dose.start - med.start)) for dose in doses if dose.sent == med.sent and abs(dose.start - med.start) <= max_window]
        if cand:
            dose = sorted(cand, key=lambda x: x[1])[0][0]
            # optional: attach nearest frequency cue
            f_cand = [(f, abs(f.start - med.start)) for f in freqs if f.sent == med.sent and abs(f.start - med.start) <= max_window]
            freq = sorted(f_cand, key=lambda x: x[1])[0][0].text if f_cand else ""
            ctx = med.sent.text
            pairs.append((med.text, dose.text, freq or ""))
    return pairs

In [ ]:
for i, txt in enumerate(emr, 1):
    doc = nlp(txt)
    neg = detect_negation(doc,NEG_CUES)
    rel = link_med_dose(doc)

    print(f"\n=== DOC {i} ===")
    print("Negation over symptoms/findings:")
    for x in neg:
        flag = "NEGADO" if x["negated"] else "AFIRMADO"
        print(f"  - {x['text']} ({x['label']}) → {flag}")

    print("\nMedication–Dose(–Freq) pairs:")
    for med, dose, freq in rel:
        print(f"  - {med} ↔ {dose}" + (f" ↔ {freq}" if freq else ""))


=== DOC 1 ===
Negation over symptoms/findings:
  - fiebre (SINTOMA) → NEGADO
  - dolor (SINTOMA) → AFIRMADO
  - disnea (SINTOMA) → AFIRMADO
  - crepitantes (HALLAZGO) → AFIRMADO

Medication–Dose(–Freq) pairs:
  - paracetamol ↔ 500 mg ↔ cada

=== DOC 2 ===
Negation over symptoms/findings:
  - tos (SINTOMA) → NEGADO
  - cefalea (SINTOMA) → NEGADO
  - Dolor (SINTOMA) → AFIRMADO
  - vómito (SINTOMA) → NEGADO

Medication–Dose(–Freq) pairs:
  - metformina ↔ 850 mg ↔ veces al día
  - Amoxicilina ↔ 500 mg

=== DOC 3 ===
Negation over symptoms/findings:
  - Faringitis (SINTOMA) → AFIRMADO
  - disnea (SINTOMA) → NEGADO
  - fiebre (SINTOMA) → NEGADO
  - Dolor (SINTOMA) → AFIRMADO
  - adenopatías (SINTOMA) → AFIRMADO

Medication–Dose(–Freq) pairs:
  - ibuprofeno ↔ 400mg ↔ cada


# Building a **Named Entity Recognition (NER)** system with Transformers

## Spanish BERT-based model from Huggingface

In [ ]:
!pip -q install transformers torch accelerate

## Load the model

In [ ]:
from transformers import pipeline

pipe = pipeline("token-classification", model="lcampillos/roberta-es-clinical-trials-ner")


In [ ]:
for i, txt in enumerate(emr, 1):
  print(f"\n=== DOC {i} ===")
  results = pipe(txt)
  for ent in results:
    text = ent["word"].replace('Ġ','')
    text = text.replace('Ã¡','á')
    text = text.replace('Ã³','ó')
    text = text.replace('ÃŃ','ó')
    print(f"{text:<15} → {ent['entity'].split('-')[1]}")


=== DOC 1 ===
fiebre          → DISO
dolor           → DISO
torácico        → DISO
disnea          → DISO
paracetamol     → CHEM
crepitantes     → DISO
base            → ANAT
pulmonar        → ANAT

=== DOC 2 ===
DM              → DISO
2               → DISO
tos             → DISO
cefalea         → DISO
metformina      → CHEM
850             → CHEM
Dolor           → DISO
abdominal       → DISO
vómito          → DISO
Amoxicilina     → CHEM
500             → CHEM

=== DOC 3 ===
Far             → DISO
ingitis         → DISO
ibuprofeno      → CHEM
disnea          → DISO
fiebre          → DISO
Dolor           → DISO
en              → DISO
garganta        → DISO
adenopatóas     → DISO
cervicales      → DISO


# Using **Large Language Models (LLMs)** with **Google Gemini**

In [ ]:
from google.colab import userdata
from google import genai

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY"

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
#client = genai.Client(api_key=userdata.get('gemini_key'))
client = genai.Client(api_key="YOUR_API_KEY")

In [ ]:
# Poner prompt en Español
context = """Given the following texts, return medical entities you could find, classify them in 4 semantic groups:
  ANAT: body parts and anatomy (e.g. garganta, 'throat')
  CHEM: chemical entities and pharmacological substances (e.g. aspirina,'aspirin')
  DISO: pathologic conditions (e.g. dolor, 'pain')
  PROC: diagnostic and therapeutic procedures, laboratory analyses and medical research activities (e.g. cirugía, 'surgery')"""

# Algunas configuraciones
safety_settings = [{"category": "HARM_CATEGORY_DANGEROUS", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}]


In [ ]:
for txt in emr:
  response = client.models.generate_content(
      model="gemini-2.5-flash",
      contents=context + '\n' + txt
  )
  print('\n\n' + response.text)



Here are the medical entities found in the text, classified into the specified semantic groups:

*   **ANAT**:
    *   base pulmonar (lung base)
*   **CHEM**:
    *   paracetamol
*   **DISO**:
    *   fiebre (fever)
    *   dolor torácico (chest pain)
    *   disnea (dyspnea)
    *   crepitantes (crackles/crepitations)
*   **PROC**:
    *   Examen (examination)


Here are the medical entities found in the texts, classified into the specified semantic groups:

*   **ANAT**:
    *   (None found)
*   **CHEM**:
    *   metformina
    *   Amoxicilina
*   **DISO**:
    *   DM2 (Diabetes Mellitus tipo 2)
    *   tos
    *   cefalea
    *   Dolor abdominal
    *   vómito
*   **PROC**:
    *   (None found)


Here are the medical entities found in the text, classified into the specified semantic groups:

**ANAT:**
*   garganta (throat)

**CHEM:**
*   ibuprofeno (ibuprofen)

**DISO:**
*   Faringitis (Pharyngitis)
*   disnea (dyspnea)
*   fiebre (fever)
*   Dolor (pain)
*   adenopatías cervicale